# Обработка датасета

## Библиотеки и скрипты

In [1]:
import numpy as np
import pandas as pd
import re
from transliterate import translit, get_available_language_codes

# Библиотека для предобработки датасетов
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
from data_preprocessing import DataPreprocessor
dp = DataPreprocessor()

# Отключаем предупреждения
import warnings
warnings.simplefilter('ignore')

In [13]:
# Получение ClientID
def get_id(x):
    id = re.search('YA:.*', x).group(0)
    return re.sub('YA:', '', id).strip()

## Обзор данных

In [2]:
# Загрузка датасета
df = pd.read_csv('../data/LEAD.csv', encoding='cp1251', sep=';')

In [3]:
df.head(5)

,ID,Статус,Название лида,Обращение,Имя,Отчество,Фамилия,Дата рождения,Дата создания,Источник,...,Наличие 1С,Причина отказа лида (список),Партнерство,Подписан на рассылку,UF_CRM_TEXTAREA,UF_CRM_TRANID,UF_CRM_FORMNAME,UF_CRM_COOKIES,UF_CRM_VASHESOOBSCHE,Отправка списка компонентов
0,10071,Некачественный лид,Алтимо UA R,NaN,Дмитрий,NaN,NaN,NaN,01.04.2021 1:01,Веб-сайт HubEx.ru,...,NaN,Повторный лид,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN
1,10069,Обработан,Алтимо UA R,NaN,Дмитрий,NaN,NaN,NaN,01.04.2021 1:00,Веб-сайт HubEx.ru,...,NaN,NaN,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN
2,10067,Обработан,One&Double,NaN,Милехина Надежда,NaN,NaN,NaN,31.03.2021 22:18,Веб-сайт HubEx.ru,...,NaN,NaN,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN
3,10065,Обработан,Тимур UA,NaN,Тимур,NaN,NaN,NaN,31.03.2021 14:52,Веб-сайт HubEx.ru,...,NaN,NaN,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN
4,10063,Обработан,Лебедев Николай Юрьевич,NaN,Лебедев Николай Юрьевич,NaN,NaN,NaN,31.03.2021 13:59,Веб-сайт HubEx.ru,...,NaN,NaN,NaN,нет,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(1407, 90)

## Обработка данных

In [5]:
df.columns

Index(['ID', 'Статус', 'Название лида', 'Обращение', 'Имя', 'Отчество',
       'Фамилия', 'Дата рождения', 'Дата создания', 'Источник',
       'Рабочий телефон', 'Мобильный телефон', 'Номер факса',
       'Домашний телефон', 'Номер пейджера', 'Телефон для рассылок',
       'Другой телефон', 'Корпоративный сайт', 'Личная страница',
       'Страница Facebook', 'Страница ВКонтакте', 'Страница LiveJournal',
       'Микроблог Twitter', 'Другой сайт', 'Рабочий e-mail', 'Частный e-mail',
       'E-mail для рассылок', 'Другой e-mail', 'Контакт Facebook',
       'Контакт Telegram', 'Контакт ВКонтакте', 'Контакт Skype',
       'Контакт Viber', 'Комментарии Instagram', 'Контакт Битрикс24.Network',
       'Онлайн-чат', 'Контакт Открытая линия', 'Контакт ICQ',
       'Контакт MSN/Live!', 'Контакт Jabber', 'Другой контакт',
       'Ответственный', 'Дополнительно о статусе',
       'Дополнительно об источнике', 'Кем создан', 'Дата изменения',
       'Кем изменен', 'Название компании', 'Должность', 'А

In [6]:
# Удаление ненужных данных
columns = ['Дата рождения', 'Рабочий телефон', 'Мобильный телефон', 'Номер факса', 'Домашний телефон', 'Номер пейджера', 
           'Телефон для рассылок', 'Другой телефон', 'Личная страница', 'Страница Facebook', 'Страница ВКонтакте', 
           'Страница LiveJournal', 'Микроблог Twitter', 'Другой сайт', 'Частный e-mail', 'E-mail для рассылок', 
           'Другой e-mail', 'Контакт Facebook', 'Контакт Telegram', 'Контакт ВКонтакте', 'Контакт Skype', 'Контакт Viber', 
           'Комментарии Instagram', 'Контакт Битрикс24.Network', 'Онлайн-чат', 'Контакт Открытая линия', 'Контакт ICQ', 
           'Контакт MSN/Live!', 'Контакт Jabber', 'Другой контакт', 'Ответственный', 'Дополнительно о статусе', 'Кем создан', 
           'Дата изменения', 'Кем изменен', 'Улица, номер дома', 'Квартира, офис, комната, этаж', 'Район', 'Регион', 
           'Почтовый индекс', 'Страна', 'Комментарий', 'Создан CRM-формой', 'Клиент', 'Путь клиента', 'Компоненты', 
           'Причина отказа сделки', 'Причина отказа сделки (список)', 'Проект', 'Конфигурация 1С', 'Наличие 1С', 
           'Причина отказа лида (список)', 'Партнерство', 'Подписан на рассылку', 'UF_CRM_TEXTAREA', 'UF_CRM_TRANID', 
           'UF_CRM_FORMNAME', 'UF_CRM_COOKIES', 'UF_CRM_VASHESOOBSCHE', 'Отправка списка компонентов']
df = dp.drop_col(df, columns)
df.head(2)

,ID,Статус,Название лида,Обращение,Имя,Отчество,Фамилия,Дата создания,Источник,Корпоративный сайт,...,UTM Medium,UTM Campaign,UTM Content,UTM Term,Классификатор отрасли,Ёмкость лицензии,Отрасль,Отрасль (список),Страница в интернете,Источник трафика
0,10071,Некачественный лид,Алтимо UA R,NaN,Дмитрий,NaN,NaN,01.04.2021 1:01,Веб-сайт HubEx.ru,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10069,Обработан,Алтимо UA R,NaN,Дмитрий,NaN,NaN,01.04.2021 1:00,Веб-сайт HubEx.ru,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,Выездное обслуживание оборудования - Спецтехника,NaN,NaN,NaN


In [7]:
# Переименование столбцов
df.columns = dp.get_translite(df.columns, 'crm_')
df.columns

Index(['crm_id', 'crm_status', 'crm_nazvanie_lida', 'crm_obraschenie',
       'crm_imja', 'crm_otchestvo', 'crm_familija', 'crm_data_sozdanija',
       'crm_istochnik', 'crm_korporativnyj_sajt', 'crm_rabochij_e_mail',
       'crm_dopolnitelno_ob_istochnike', 'crm_nazvanie_kompanii',
       'crm_dolzhnost', 'crm_adres', 'crm_naselennyj_punkt', 'crm_summa',
       'crm_valjuta', 'crm_povtornyj_lid', 'crm_utm_source', 'crm_utm_medium',
       'crm_utm_campaign', 'crm_utm_content', 'crm_utm_term',
       'crm_klassifikator_otrasli', 'crm_emkost_litsenzii', 'crm_otrasl',
       'crm_otrasl_spisok', 'crm_stranitsa_v_internete',
       'crm_istochnik_trafika'],
      dtype='object')

In [8]:
# Убираем 2020 год (там одни тесты)
# df = df[df['crm_data_sozdanija'].str.contains('.*2021.*')]

**Обработка пропусков**

In [9]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1407 entries, 0 to 1406
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   crm_id                          1407 non-null   int64  
 1   crm_status                      1407 non-null   object 
 2   crm_nazvanie_lida               1407 non-null   object 
 3   crm_obraschenie                 330 non-null    object 
 4   crm_imja                        1280 non-null   object 
 5   crm_otchestvo                   47 non-null     object 
 6   crm_familija                    177 non-null    object 
 7   crm_data_sozdanija              1407 non-null   object 
 8   crm_istochnik                   1399 non-null   object 
 9   crm_korporativnyj_sajt          149 non-null    object 
 10  crm_rabochij_e_mail             1242 non-null   object 
 11  crm_dopolnitelno_ob_istochnike  1221 non-null   object 
 12  crm_nazvanie_kompanii           10

crm_id                               0
crm_status                           0
crm_nazvanie_lida                    0
crm_obraschenie                   1077
crm_imja                           127
crm_otchestvo                     1360
crm_familija                      1230
crm_data_sozdanija                   0
crm_istochnik                        8
crm_korporativnyj_sajt            1258
crm_rabochij_e_mail                165
crm_dopolnitelno_ob_istochnike     186
crm_nazvanie_kompanii              390
crm_dolzhnost                     1064
crm_adres                         1332
crm_naselennyj_punkt              1343
crm_summa                            0
crm_valjuta                          0
crm_povtornyj_lid                    0
crm_utm_source                    1301
crm_utm_medium                    1302
crm_utm_campaign                  1301
crm_utm_content                   1305
crm_utm_term                      1391
crm_klassifikator_otrasli         1153
crm_emkost_litsenzii     

In [10]:
# Корректировка значений
df.loc[df['crm_obraschenie'].isnull(), 'crm_obraschenie'] = 'undefined'
df.loc[df['crm_imja'].isnull(), 'crm_imja'] = 'undefined'
df.loc[df['crm_otchestvo'].isnull(), 'crm_otchestvo'] = 'undefined'
df.loc[df['crm_familija'].isnull(), 'crm_familija'] = 'undefined'
df.loc[df['crm_istochnik'].isnull(), 'crm_istochnik'] = 'undefined'
df.loc[df['crm_korporativnyj_sajt'].isnull(), 'crm_korporativnyj_sajt'] = 'undefined'
df.loc[df['crm_rabochij_e_mail'].isnull(), 'crm_rabochij_e_mail'] = 'undefined'
df.loc[df['crm_dopolnitelno_ob_istochnike'].isnull(), 'crm_dopolnitelno_ob_istochnike'] = 'undefined' # Для регулярки
df.loc[df['crm_nazvanie_kompanii'].isnull(), 'crm_nazvanie_kompanii'] = 'undefined'
df.loc[df['crm_dolzhnost'].isnull(), 'crm_dolzhnost'] = 'undefined'
df.loc[df['crm_adres'].isnull(), 'crm_adres'] = 'undefined'
df.loc[df['crm_naselennyj_punkt'].isnull(), 'crm_naselennyj_punkt'] = 'undefined'
df.loc[df['crm_utm_source'].isnull(), 'crm_utm_source'] = 'undefined'
df.loc[df['crm_utm_medium'].isnull(), 'crm_utm_medium'] = 'undefined'
df.loc[df['crm_utm_campaign'].isnull(), 'crm_utm_campaign'] = 'undefined'
df.loc[df['crm_utm_content'].isnull(), 'crm_utm_content'] = 'undefined'
df.loc[df['crm_utm_term'].isnull(), 'crm_utm_term'] = 'undefined'
df.loc[df['crm_klassifikator_otrasli'].isnull(), 'crm_klassifikator_otrasli'] = 'undefined'
df.loc[df['crm_emkost_litsenzii'].isnull(), 'crm_emkost_litsenzii'] = df['crm_emkost_litsenzii'].median()
df.loc[df['crm_otrasl'].isnull(), 'crm_otrasl'] = 'undefined'
df.loc[df['crm_otrasl_spisok'].isnull(), 'crm_otrasl_spisok'] = 'undefined'
df.loc[df['crm_stranitsa_v_internete'].isnull(), 'crm_stranitsa_v_internete'] = 'undefined'
df.loc[df['crm_istochnik_trafika'].isnull(), 'crm_istochnik_trafika'] = 'undefined'

In [14]:
# Получаем ClientID
df['crm_client_id'] = df['crm_dopolnitelno_ob_istochnike'].apply(lambda x: get_id(x) if 'YA' in x else 'undefined', 1)
df.loc[(df['crm_client_id'].isnull()) | (df['crm_client_id'] == ''), 'crm_client_id'] = 'undefined'

In [15]:
df.head(5)

,crm_id,crm_status,crm_nazvanie_lida,crm_obraschenie,crm_imja,crm_otchestvo,crm_familija,crm_data_sozdanija,crm_istochnik,crm_korporativnyj_sajt,...,crm_utm_campaign,crm_utm_content,crm_utm_term,crm_klassifikator_otrasli,crm_emkost_litsenzii,crm_otrasl,crm_otrasl_spisok,crm_stranitsa_v_internete,crm_istochnik_trafika,crm_client_id
0,10071,Некачественный лид,Алтимо UA R,undefined,Дмитрий,undefined,undefined,01.04.2021 1:01,Веб-сайт HubEx.ru,undefined,...,undefined,undefined,undefined,undefined,10.0,undefined,undefined,undefined,undefined,1615817020919653525
1,10069,Обработан,Алтимо UA R,undefined,Дмитрий,undefined,undefined,01.04.2021 1:00,Веб-сайт HubEx.ru,undefined,...,undefined,undefined,undefined,undefined,4.0,Выездное обслуживание оборудования - Спецтехника,undefined,undefined,undefined,1615817020919653525
2,10067,Обработан,One&Double,undefined,Милехина Надежда,undefined,undefined,31.03.2021 22:18,Веб-сайт HubEx.ru,undefined,...,undefined,undefined,undefined,undefined,10.0,undefined,undefined,undefined,Контекстная реклама,1617218182652379262
3,10065,Обработан,Тимур UA,undefined,Тимур,undefined,undefined,31.03.2021 14:52,Веб-сайт HubEx.ru,undefined,...,undefined,undefined,undefined,undefined,10.0,undefined,undefined,undefined,Поиск Google,undefined
4,10063,Обработан,Лебедев Николай Юрьевич,undefined,Лебедев Николай Юрьевич,undefined,undefined,31.03.2021 13:59,Веб-сайт HubEx.ru,undefined,...,undefined,undefined,undefined,undefined,10.0,undefined,undefined,undefined,undefined,1617186237647777106


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1407 entries, 0 to 1406
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   crm_id                          1407 non-null   int64  
 1   crm_status                      1407 non-null   object 
 2   crm_nazvanie_lida               1407 non-null   object 
 3   crm_obraschenie                 1407 non-null   object 
 4   crm_imja                        1407 non-null   object 
 5   crm_otchestvo                   1407 non-null   object 
 6   crm_familija                    1407 non-null   object 
 7   crm_data_sozdanija              1407 non-null   object 
 8   crm_istochnik                   1407 non-null   object 
 9   crm_korporativnyj_sajt          1407 non-null   object 
 10  crm_rabochij_e_mail             1407 non-null   object 
 11  crm_dopolnitelno_ob_istochnike  1407 non-null   object 
 12  crm_nazvanie_kompanii           14

In [17]:
# Выводим имена столцов
dp.get_idx_col(df.columns)

0 crm_id
1 crm_status
2 crm_nazvanie_lida
3 crm_obraschenie
4 crm_imja
5 crm_otchestvo
6 crm_familija
7 crm_data_sozdanija
8 crm_istochnik
9 crm_korporativnyj_sajt
10 crm_rabochij_e_mail
11 crm_dopolnitelno_ob_istochnike
12 crm_nazvanie_kompanii
13 crm_dolzhnost
14 crm_adres
15 crm_naselennyj_punkt
16 crm_summa
17 crm_valjuta
18 crm_povtornyj_lid
19 crm_utm_source
20 crm_utm_medium
21 crm_utm_campaign
22 crm_utm_content
23 crm_utm_term
24 crm_klassifikator_otrasli
25 crm_emkost_litsenzii
26 crm_otrasl
27 crm_otrasl_spisok
28 crm_stranitsa_v_internete
29 crm_istochnik_trafika
30 crm_client_id
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


In [18]:
# Новый порядок столбцов
new_order = [30, 0, 7, 1, 2, 3, 4, 5, 6, 13, 12, 8, 9, 10, 11, 14, 15, 28, 29, 25, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27]
df = df[df.columns[new_order]]

In [19]:
df.columns

Index(['crm_client_id', 'crm_id', 'crm_data_sozdanija', 'crm_status',
       'crm_nazvanie_lida', 'crm_obraschenie', 'crm_imja', 'crm_otchestvo',
       'crm_familija', 'crm_dolzhnost', 'crm_nazvanie_kompanii',
       'crm_istochnik', 'crm_korporativnyj_sajt', 'crm_rabochij_e_mail',
       'crm_dopolnitelno_ob_istochnike', 'crm_adres', 'crm_naselennyj_punkt',
       'crm_stranitsa_v_internete', 'crm_istochnik_trafika',
       'crm_emkost_litsenzii', 'crm_summa', 'crm_valjuta', 'crm_povtornyj_lid',
       'crm_utm_source', 'crm_utm_medium', 'crm_utm_campaign',
       'crm_utm_content', 'crm_utm_term', 'crm_klassifikator_otrasli',
       'crm_otrasl', 'crm_otrasl_spisok'],
      dtype='object')

In [20]:
# Приведение типов
df = dp.astype_col(df, ['crm_emkost_litsenzii'], coltype='uint8')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1407 entries, 0 to 1406
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   crm_client_id                   1407 non-null   object 
 1   crm_id                          1407 non-null   int64  
 2   crm_data_sozdanija              1407 non-null   object 
 3   crm_status                      1407 non-null   object 
 4   crm_nazvanie_lida               1407 non-null   object 
 5   crm_obraschenie                 1407 non-null   object 
 6   crm_imja                        1407 non-null   object 
 7   crm_otchestvo                   1407 non-null   object 
 8   crm_familija                    1407 non-null   object 
 9   crm_dolzhnost                   1407 non-null   object 
 10  crm_nazvanie_kompanii           1407 non-null   object 
 11  crm_istochnik                   1407 non-null   object 
 12  crm_korporativnyj_sajt          14

**Основные метрики числовых данных**

In [22]:
df.describe()

,crm_id,crm_emkost_litsenzii,crm_summa
count,1407.000000,1407.000000,1.407000e+03
mean,4867.312011,14.525942,1.077626e+04
std,2878.472913,23.801258,8.391499e+04
min,519.000000,0.000000,0.000000e+00
25%,2770.000000,10.000000,0.000000e+00
50%,4335.000000,10.000000,0.000000e+00
75%,7065.000000,10.000000,0.000000e+00
max,10071.000000,250.000000,1.512000e+06


**Удаление дубликатов**

In [23]:
# Удаление дубликатов по crm_id
df = df.drop_duplicates(subset='crm_id', keep="first")

In [24]:
df.head(5)

,crm_client_id,crm_id,crm_data_sozdanija,crm_status,crm_nazvanie_lida,crm_obraschenie,crm_imja,crm_otchestvo,crm_familija,crm_dolzhnost,...,crm_valjuta,crm_povtornyj_lid,crm_utm_source,crm_utm_medium,crm_utm_campaign,crm_utm_content,crm_utm_term,crm_klassifikator_otrasli,crm_otrasl,crm_otrasl_spisok
0,1615817020919653525,10071,01.04.2021 1:01,Некачественный лид,Алтимо UA R,undefined,Дмитрий,undefined,undefined,Директор,...,Рубль,N,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined
1,1615817020919653525,10069,01.04.2021 1:00,Обработан,Алтимо UA R,undefined,Дмитрий,undefined,undefined,Директор,...,Рубль,N,undefined,undefined,undefined,undefined,undefined,undefined,Выездное обслуживание оборудования - Спецтехника,undefined
2,1617218182652379262,10067,31.03.2021 22:18,Обработан,One&Double,undefined,Милехина Надежда,undefined,undefined,Руководитель,...,Рубль,N,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined
3,undefined,10065,31.03.2021 14:52,Обработан,Тимур UA,undefined,Тимур,undefined,undefined,undefined,...,Рубль,N,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined
4,1617186237647777106,10063,31.03.2021 13:59,Обработан,Лебедев Николай Юрьевич,undefined,Лебедев Николай Юрьевич,undefined,undefined,undefined,...,Рубль,N,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined


In [25]:
df.shape

(1407, 31)

## Сохранение данных

In [26]:
# Сохранение в файл
df.to_csv('../data/td_crm.csv', index=False, encoding='utf-8', sep=';')